In [1]:
from dotenv import load_dotenv
load_dotenv()


True

CREATE DATA ON LOCAL

In [2]:

import pandas as pd

# QA
inputs = [
    "For customer-facing applications, which company's models dominate the top rankings?",
    "What percentage of respondents are using RAG in some form?",
    "How often are most respondents updating their models?",
]

outputs = [
    "OpenAI models dominate, with 3 of the top 5 and half of the top 10 most popular models for customer-facing apps.",
    "70% of respondents are using RAG in some form.",
    "More than 50% update their models at least monthly, with 17% doing so weekly.",
]

# Dataset
qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]
df = pd.DataFrame(qa_pairs)

# Write to csv
csv_path = "../data/goldens.csv"
df.to_csv(csv_path, index=False)

ADD DATA TO LANGSMITH - to evaluate the RAG APP

In [3]:
from langsmith import Client

client = Client()
dataset_name = "LLMOPS_Dataset"

# Store
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Input and expected output pairs for LLMOPS evaluations",
)
client.create_examples(
    inputs=[{"question": q} for q in inputs],
    outputs=[{"answer": a} for a in outputs],
    dataset_id=dataset.id,
)

{'example_ids': ['43edae04-5b8d-4824-89c4-f011d7e3ff85',
  '9820c451-b433-4091-b79a-4042b2244967',
  'dfa4efb5-5bd9-4a90-9b49-459b6e4a0646'],
 'count': 3}

Run The RAG APP

In [4]:
import sys
sys.path.append("..")  # Add parent directory to path to find multi_doc_chat module

from pathlib import Path
from multi_doc_chat.src.document_ingestion.data_ingestion import ChatIngestor
from multi_doc_chat.src.document_chat.retrieval import ConversationalRAG
import os

# Simple file adapter for local file paths
class LocalFileAdapter:
    """Adapter for local file paths to work with ChatIngestor."""
    def __init__(self, file_path: str):
        self.path = Path(file_path)
        self.name = self.path.name
    
    def getbuffer(self) -> bytes:
        return self.path.read_bytes()


def answer_ai_report_question(
    inputs: dict,
    data_path: str = "../data/The 2025 AI Engineering Report.txt",
    chunk_size: int = 1000,
    chunk_overlap: int = 200,
    k: int = 5
) -> dict:
    """
    Answer questions about the AI Engineering Report using RAG.
    
    Args:
        inputs: Dictionary containing the question, e.g., {"question": "What is RAG?"}
        data_path: Path to the AI Engineering Report text file
        chunk_size: Size of text chunks for splitting
        chunk_overlap: Overlap between chunks
        k: Number of documents to retrieve
    
    Returns:
        Dictionary with the answer, e.g., {"answer": "RAG stands for..."}
    """
    try:
        # Extract question from inputs
        question = inputs.get("question", "")
        if not question:
            return {"answer": "No question provided"}
        
        # Check if file exists
        if not Path(data_path).exists():
            return {"answer": f"Data file not found: {data_path}"}
        
        # Create file adapter
        file_adapter = LocalFileAdapter(data_path)
        
        # Build index using ChatIngestor
        ingestor = ChatIngestor(
            temp_base="data",
            faiss_base="faiss_index",
            use_session_dirs=True
        )
        
        # Build retriever
        ingestor.built_retriver(
            uploaded_files=[file_adapter],
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            k=k
        )
        
        # Get session ID and index path
        session_id = ingestor.session_id
        index_path = f"faiss_index/{session_id}"
        
        # Create RAG instance and load retriever
        rag = ConversationalRAG(session_id=session_id)
        rag.load_retriever_from_faiss(
            index_path=index_path,
            k=k,
            index_name=os.getenv("FAISS_INDEX_NAME", "index")
        )
        
        # Get answer
        answer = rag.invoke(question, chat_history=[])
        
        return {"answer": answer}
        
    except Exception as e:
        return {"answer": f"Error: {str(e)}"}

In [5]:
# Test the function with a sample question
test_input = {"question": "For customer-facing applications, which company's models dominate the top rankings?"}
result = answer_ai_report_question(test_input)
print("Question:", test_input["question"])
print("\nAnswer:", result["answer"])

{"timestamp": "2026-01-04T13:14:38.581178Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2026-01-04T13:14:38.581726Z", "level": "info", "event": "Loaded GROQ_API_KEY from individual env var"}
{"timestamp": "2026-01-04T13:14:38.582159Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GROQ_API_KEY": "gsk_1b...", "GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2026-01-04T13:14:38.582751Z", "level": "info", "event": "API keys loaded"}
{"config_keys": ["embedding_model", "retriever", "llm"], "timestamp": "2026-01-04T13:14:38.586562Z", "level": "info", "event": "YAML config loaded"}
{"session_id": "session_20260104_171438_7a3f5797", "temp_dir": "data/session_20260104_171438_7a3f5797", "faiss_dir": "faiss_index/session_20260104_171438_7a3f5797", "sessionized": true, "timestamp": "2026-01-04T13:14:38.587624Z", "level": "info", "event": "ChatIngestor initialized"}
{"uploaded": "The 2025 AI Engineering Report.txt", "saved_as

Question: For customer-facing applications, which company's models dominate the top rankings?

Answer: OpenAI’s models dominate the top rankings for customer‑facing applications.


In [6]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

In [7]:
# Example: Test with all golden questions
print("Testing all questions from the dataset:\n")
for i, q in enumerate(inputs, 1):
    test_input = {"question": q}
    result = answer_ai_report_question(test_input)
    print(f"Q{i}: {q}")
    print(f"A{i}: {result['answer']}\n")
    print("-" * 80 + "\n")

{"timestamp": "2026-01-04T13:16:37.527733Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2026-01-04T13:16:37.528284Z", "level": "info", "event": "Loaded GROQ_API_KEY from individual env var"}
{"timestamp": "2026-01-04T13:16:37.528681Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GROQ_API_KEY": "gsk_1b...", "GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2026-01-04T13:16:37.529158Z", "level": "info", "event": "API keys loaded"}
{"config_keys": ["embedding_model", "retriever", "llm"], "timestamp": "2026-01-04T13:16:37.532419Z", "level": "info", "event": "YAML config loaded"}
{"session_id": "session_20260104_171637_ed860953", "temp_dir": "data/session_20260104_171637_ed860953", "faiss_dir": "faiss_index/session_20260104_171637_ed860953", "sessionized": true, "timestamp": "2026-01-04T13:16:37.534016Z", "level": "info", "event": "ChatIngestor initialized"}
{"uploaded": "The 2025 AI Engineering Report.txt", "saved_as

Testing all questions from the dataset:



{"added": 1, "index": "faiss_index/session_20260104_171637_ed860953", "timestamp": "2026-01-04T13:16:41.099349Z", "level": "info", "event": "FAISS index updated"}
{"k": 5, "fetch_k": 20, "lambda_mult": 0.5, "timestamp": "2026-01-04T13:16:41.099751Z", "level": "info", "event": "Using MMR search"}
{"timestamp": "2026-01-04T13:16:41.100825Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2026-01-04T13:16:41.101049Z", "level": "info", "event": "Loaded GROQ_API_KEY from individual env var"}
{"timestamp": "2026-01-04T13:16:41.101282Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GROQ_API_KEY": "gsk_1b...", "GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2026-01-04T13:16:41.101541Z", "level": "info", "event": "API keys loaded"}
{"config_keys": ["embedding_model", "retriever", "llm"], "timestamp": "2026-01-04T13:16:41.102943Z", "level": "info", "event": "YAML config loaded"}
{"provider": "groq", "model": "openai/gpt-oss-2

Q1: For customer-facing applications, which company's models dominate the top rankings?
A1: OpenAI’s models dominate the top rankings for customer‑facing applications.

--------------------------------------------------------------------------------



{"added": 1, "index": "faiss_index/session_20260104_171645_460d953b", "timestamp": "2026-01-04T13:16:48.475303Z", "level": "info", "event": "FAISS index updated"}
{"k": 5, "fetch_k": 20, "lambda_mult": 0.5, "timestamp": "2026-01-04T13:16:48.475832Z", "level": "info", "event": "Using MMR search"}
{"timestamp": "2026-01-04T13:16:48.477255Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2026-01-04T13:16:48.477615Z", "level": "info", "event": "Loaded GROQ_API_KEY from individual env var"}
{"timestamp": "2026-01-04T13:16:48.477937Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GROQ_API_KEY": "gsk_1b...", "GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2026-01-04T13:16:48.478347Z", "level": "info", "event": "API keys loaded"}
{"config_keys": ["embedding_model", "retriever", "llm"], "timestamp": "2026-01-04T13:16:48.480234Z", "level": "info", "event": "YAML config loaded"}
{"provider": "groq", "model": "openai/gpt-oss-2

Q2: What percentage of respondents are using RAG in some form?
A2: 70% of respondents are using RAG in some form.

--------------------------------------------------------------------------------



{"added": 1, "index": "faiss_index/session_20260104_171652_5a603a38", "timestamp": "2026-01-04T13:16:56.091058Z", "level": "info", "event": "FAISS index updated"}
{"k": 5, "fetch_k": 20, "lambda_mult": 0.5, "timestamp": "2026-01-04T13:16:56.091520Z", "level": "info", "event": "Using MMR search"}
{"timestamp": "2026-01-04T13:16:56.092788Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2026-01-04T13:16:56.093098Z", "level": "info", "event": "Loaded GROQ_API_KEY from individual env var"}
{"timestamp": "2026-01-04T13:16:56.093414Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GROQ_API_KEY": "gsk_1b...", "GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2026-01-04T13:16:56.093707Z", "level": "info", "event": "API keys loaded"}
{"config_keys": ["embedding_model", "retriever", "llm"], "timestamp": "2026-01-04T13:16:56.095691Z", "level": "info", "event": "YAML config loaded"}
{"provider": "groq", "model": "openai/gpt-oss-2

Q3: How often are most respondents updating their models?
A3: Most respondents update their models on a monthly basis.

--------------------------------------------------------------------------------



Prebuilt Evaluator

In [ ]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator

#  Bydefault LangChainStringEvaluator use OpenAI  so below code will give an error without OpenAI API key.
# Evaluators
qa_evaluator = [LangChainStringEvaluator("cot_qa")]
dataset_name = "LLMOPS_Dataset"

# Run evaluation using our RAG function
experiment_results = evaluate(
    answer_ai_report_question,
    data=dataset_name,
    evaluators=qa_evaluator,
    experiment_prefix="test-agenticAIReport-qa-rag",
    # Experiment metadata
    metadata={
        "variant": "RAG with FAISS and AI Engineering Report",
        "chunk_size": 1000,
        "chunk_overlap": 200,
        "k": 5,
    },
)

E0000 00:00:1761230954.908546  392834 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` b

View the evaluation results for experiment: 'test-agenticAIReport-qa-rag-gemini-85cbad80' at:
https://smith.langchain.com/o/5ab3a087-cdc4-52b4-b7ec-104866d876b9/datasets/0906187c-3089-4bae-98a1-bf67dba8caf6/compare?selectedSessions=c0917fa7-950b-49d8-ad0b-1298e7f4f9e2




0it [00:00, ?it/s]{"timestamp": "2025-10-23T14:49:17.249961Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2025-10-23T14:49:17.250718Z", "level": "info", "event": "Loaded GROQ_API_KEY from individual env var"}
{"timestamp": "2025-10-23T14:49:17.251276Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GROQ_API_KEY": "gsk_1b...", "GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2025-10-23T14:49:17.251777Z", "level": "info", "event": "API keys loaded"}
{"timestamp": "2025-10-23T14:49:17.249961Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2025-10-23T14:49:17.250718Z", "level": "info", "event": "Loaded GROQ_API_KEY from individual env var"}
{"timestamp": "2025-10-23T14:49:17.251276Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GROQ_API_KEY": "gsk_1b...", "GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2025-10-23T14:49:17.251777Z", "level": "inf

## Custom Correctness Evaluator

Creating an LLM-as-a-Judge evaluator to assess semantic and factual alignment



In [8]:
from langsmith.schemas import Run, Example
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

def correctness_evaluator(run: Run, example: Example) -> dict:
    """
    Custom LLM-as-a-Judge evaluator for correctness.
    
    Correctness means how well the actual model output matches the reference output 
    in terms of factual accuracy, coverage, and meaning.
    
    Args:
        run: The Run object containing the actual outputs
        example: The Example object containing the expected outputs
    
    Returns:
        dict with 'score' (1 for correct, 0 for incorrect) and 'reasoning'
    """
    # Extract actual and expected outputs
    actual_output = run.outputs.get("answer", "")
    expected_output = example.outputs.get("answer", "")
    input_question = example.inputs.get("question", "")
    
    # Define the evaluation prompt
    eval_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an evaluator whose job is to judge correctness.

Correctness means how well the actual model output matches the reference output in terms of factual accuracy, coverage, and meaning.

- If the actual output matches the reference output semantically (even if wording differs), it should be marked correct.
- If the output misses key facts, introduces contradictions, or is factually incorrect, it should be marked incorrect.

Do not penalize for stylistic or formatting differences unless they change meaning."""),
        ("human", """<example>
<input>
{input}
</input>

<output>
Expected Output: {expected_output}

Actual Output: {actual_output}
</output>
</example>

Please grade the following agent run given the input, expected output, and actual output.
Focus only on correctness (semantic and factual alignment).

Respond with:
1. A brief reasoning (1-2 sentences)
2. A final verdict: either "CORRECT" or "INCORRECT"

Format your response as:
Reasoning: [your reasoning]
Verdict: [CORRECT or INCORRECT]""")
    ])
    
    # Initialize LLM (using Gemini as shown in your config)
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-pro",
        temperature=0
    )
    
    # Create chain and invoke
    chain = eval_prompt | llm
    
    try:
        response = chain.invoke({
            "input": input_question,
            "expected_output": expected_output,
            "actual_output": actual_output
        })
        
        response_text = response.content
        
        # Parse the response
        reasoning = ""
        verdict = ""
        
        for line in response_text.split('\n'):
            if line.startswith("Reasoning:"):
                reasoning = line.replace("Reasoning:", "").strip()
            elif line.startswith("Verdict:"):
                verdict = line.replace("Verdict:", "").strip()
        
        # Convert verdict to score (1 for correct, 0 for incorrect)
        score = 1 if "CORRECT" in verdict.upper() else 0
        
        return {
            "key": "correctness",
            "score": score,
            "reasoning": reasoning,
            "comment": f"Verdict: {verdict}"
        }
        
    except Exception as e:
        return {
            "key": "correctness",
            "score": 0,
            "reasoning": f"Error during evaluation: {str(e)}"
        }

## Run Evaluation with Custom Correctness Evaluator

In [9]:
# Run evaluation with the custom correctness evaluator
from langsmith.evaluation import evaluate

# Define evaluators - using custom correctness evaluator
evaluators = [correctness_evaluator]

dataset_name = "LLMOPS_Dataset"

# Run evaluation
experiment_results = evaluate(
    answer_ai_report_question,
    data=dataset_name,
    evaluators=evaluators,
    experiment_prefix="agenticAIReport-correctness-eval",
    description="Evaluating RAG system with custom correctness evaluator (LLM-as-a-Judge)",
    metadata={
        "variant": "RAG with FAISS and AI Engineering Report",
        "evaluator": "custom_correctness_llm_judge",
        "model": "gemini-2.5-pro",
        "chunk_size": 1000,
        "chunk_overlap": 200,
        "k": 5,
    },
)
print("\nEvaluation completed! Check the LangSmith UI for detailed results.")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

View the evaluation results for experiment: 'agenticAIReport-correctness-eval-d0c3e72e' at:
https://smith.langchain.com/o/5ab3a087-cdc4-52b4-b7ec-104866d876b9/datasets/a27d83f2-be8c-49f5-9106-6f38cb260c7d/compare?selectedSessions=76c6d724-0dba-4103-a0ab-65e84c55199a




0it [00:00, ?it/s]{"timestamp": "2026-01-04T13:47:25.932422Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2026-01-04T13:47:25.933200Z", "level": "info", "event": "Loaded GROQ_API_KEY from individual env var"}
{"timestamp": "2026-01-04T13:47:25.933781Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GROQ_API_KEY": "gsk_1b...", "GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2026-01-04T13:47:25.934343Z", "level": "info", "event": "API keys loaded"}
{"timestamp": "2026-01-04T13:47:25.932422Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2026-01-04T13:47:25.933200Z", "level": "info", "event": "Loaded GROQ_API_KEY from individual env var"}
{"timestamp": "2026-01-04T13:47:25.933781Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GROQ_API_KEY": "gsk_1b...", "GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2026-01-04T13:47:25.934343Z", "level": "inf


Evaluation completed! Check the LangSmith UI for detailed results.


## Optional: Combine Multiple Evaluators
You can use multiple evaluators together to get different perspectives on your RAG system's performance.

In [ ]:
# Example: Combine custom correctness evaluator with LangChain's built-in evaluators
from langsmith.evaluation import evaluate, LangChainStringEvaluator

# Combine custom and built-in evaluators
combined_evaluators = [
    correctness_evaluator,  # Custom LLM-as-a-Judge
    LangChainStringEvaluator("cot_qa"),  # Chain-of-thought QA evaluator
]

# Run evaluation with multiple evaluators
# Uncomment to run:
# experiment_results_combined = evaluate(
#     answer_ai_report_question,
#     data=dataset_name,
#     evaluators=combined_evaluators,
#     experiment_prefix="agenticAIReport-multi-eval",
#     description="Evaluating RAG system with multiple evaluators",
#     metadata={
#         "variant": "RAG with FAISS",
#         "evaluators": "correctness + cot_qa",
#         "chunk_size": 1000,
#         "chunk_overlap": 200,
#         "k": 5,
#     },
# )